In [1]:
# 모듈 임포트
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib
import os

In [2]:
# 피처 엔지니어링 함수 정의
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

In [3]:
# 피처 목록 정의
features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']

In [4]:
# 테스트셋 로드 및 피처 생성
test_df = pd.read_csv('../realtime/test/1.csv')  # ← 경로 확인 필요
test_df = add_features(test_df).dropna()

X_test = test_df[features].values
y_true = test_df['label'].values

In [5]:
# 스케일러 및 모델 로드
model = joblib.load('./model/one_class_svm_model.joblib')
scaler = joblib.load('./model/scaler.joblib')

X_test_scaled = scaler.transform(X_test)

In [6]:
# 예측 (One-Class SVM은 이상이면 -1, 정상은 1 반환)
y_pred_raw = model.predict(X_test_scaled)
y_pred = np.where(y_pred_raw == 1, 0, 1)  # 이상: 1 (arc), 정상: 0

# 성능 평가
print("📊 Classification Report (One-Class SVM)")
print(classification_report(y_true, y_pred, digits=4))

📊 Classification Report (One-Class SVM)
              precision    recall  f1-score   support

           0     0.9990    0.9584    0.9783     27000
           1     0.0044    0.1667    0.0086        30

    accuracy                         0.9575     27030
   macro avg     0.5017    0.5625    0.4935     27030
weighted avg     0.9979    0.9575    0.9772     27030

